In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # loads OPENAI_API_KEY from .env
client = OpenAI()

def stream_chat(messages, model="gpt-4o-mini", **kwargs):
    """
    messages = [
      {"role":"system","content":"You are concise."},
      {"role":"user","content":"Explain streams briefly."}
    ]
    """
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        **kwargs,
    )
    for chunk in resp:
        if chunk.choices and chunk.choices[0].delta and chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content



In [7]:
msgs = [
        {"role": "system", "content": "You are concise."},
        {"role": "user", "content": "Give me 3 facts about streaming."},
    ]
for token in stream_chat(msgs, temperature=0.3):
    print(token, end="", flush=True)
print()

1. **Growth in Popularity**: Streaming services have seen exponential growth, with platforms like Netflix, Amazon Prime, and Disney+ amassing millions of subscribers worldwide, significantly impacting traditional cable TV viewership.

2. **Content Variety**: Streaming platforms offer a diverse range of content, including movies, TV shows, documentaries, and original programming, catering to various audience preferences and demographics.

3. **Accessibility**: Streaming allows users to access content on multiple devices, such as smartphones, tablets, smart TVs, and computers, providing flexibility in viewing anytime and anywhere with an internet connection.


In [9]:
#!/usr/bin/env python3
from __future__ import annotations

import json
import os
import uuid
from dataclasses import dataclass, asdict
from typing import Iterable, List, Literal, Dict, Any, Optional

from dotenv import load_dotenv
from openai import OpenAI

# --- env / client ------------------------------------------------------------
load_dotenv()  # expects OPENAI_API_KEY in .env or environment
client = OpenAI()

Role = Literal["system", "user", "assistant"]


@dataclass
class Message:
    role: Role
    content: str


@dataclass
class Conversation:
    id: str
    messages: List[Message]


# --- JSON storage ------------------------------------------------------------
class JsonStore:
    def __init__(self, root: str = "./chats") -> None:
        self.root = root
        os.makedirs(self.root, exist_ok=True)

    def _path(self, cid: str) -> str:
        return os.path.join(self.root, f"{cid}.json")

    def save(self, conv: Conversation) -> None:
        data = {
            "id": conv.id,
            "messages": [asdict(m) for m in conv.messages],
        }
        with open(self._path(conv.id), "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def load(self, conversation_id: str) -> Optional[Conversation]:
        path = self._path(conversation_id)
        if not os.path.isfile(path):
            return None
        with open(path, "r", encoding="utf-8") as f:
            raw = json.load(f)
        return Conversation(
            id=raw["id"],
            messages=[Message(**m) for m in raw.get("messages", [])],
        )


# --- OpenAI streaming --------------------------------------------------------
def stream_chat(messages: List[Dict[str, str]],
                model: str = "gpt-4o-mini",
                **kwargs) -> Iterable[str]:
    """
    Yield assistant text deltas (tokens/fragments) as they arrive.
    `messages` is a list of dicts [{role, content}, ...].
    """
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        **kwargs,
    )
    for chunk in resp:
        if chunk.choices:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                yield delta.content


# --- High-level helper to send a message, stream, and persist ----------------
def send_and_stream_with_persist(store: JsonStore,
                                 conversation_id: Optional[str],
                                 user_text: str,
                                 system_if_new: Optional[str] = None,
                                 model: str = "gpt-4o-mini",
                                 **kwargs) -> Iterable[str]:
    """
    - If `conversation_id` is None or not found → create new conversation (uuid).
      Optionally bootstrap with a system message (`system_if_new`).
    - Append the user's message.
    - Stream assistant reply; yield chunks.
    - Save the final assistant message and updated history to JSON.
    Returns a generator of chunks; also returns `conversation_id` via attribute.
    """
    # Load or create conversation
    conv = None
    if conversation_id:
        conv = store.load(conversation_id)

    if conv is None:
        conversation_id = uuid.uuid4().hex
        conv = Conversation(id=conversation_id, messages=[])
        if system_if_new:  # optional system bootstrap
            conv.messages.append(Message(role="system", content=system_if_new))
        store.save(conv)  # persist immediately so the file exists

    # Append user message and persist
    conv.messages.append(Message(role="user", content=user_text))
    store.save(conv)

    # Build wire messages for OpenAI
    wire = [{"role": m.role, "content": m.content} for m in conv.messages]

    # Stream assistant reply
    assembled: List[str] = []
    try:
        for tok in stream_chat(wire, model=model, **kwargs):
            assembled.append(tok)
            yield tok
    finally:
        # Persist assistant message even if stream/error interrupted
        full = "".join(assembled)
        conv.messages.append(Message(role="assistant", content=full))
        store.save(conv)

# Convenience alias on the generator to access the id during iteration:
class StreamWithId:
    def __init__(self, gen: Iterable[str], conversation_id: str) -> None:
        self._gen = gen
        self.conversation_id = conversation_id
    def __iter__(self):
        return iter(self._gen)


def start_streaming_message(store: JsonStore,
                            user_text: str,
                            conversation_id: Optional[str] = None,
                            system_if_new: Optional[str] = None,
                            model: str = "gpt-4o-mini",
                            **kwargs) -> StreamWithId:
    """
    Wrapper that returns a generator *plus* exposes conversation_id immediately.
    """
    # Peek/create the conversation id consistently with the helper above
    if conversation_id and store.load(conversation_id) is not None:
        cid = conversation_id
    else:
        cid = conversation_id or uuid.uuid4().hex
    gen = send_and_stream_with_persist(
        store=store,
        conversation_id=cid,
        user_text=user_text,
        system_if_new=system_if_new,
        model=model,
        **kwargs,
    )
    return StreamWithId(gen, cid)




In [13]:
store = JsonStore("./chats")

stream1 = start_streaming_message(
        store,
        user_text="Explain step 2 ",
        conversation_id="62d8365610f742b9866db20f1c593487",  # new convo
        system_if_new="you are concise.",
        temperature=0.3,
    )
print(f"\nConversation ID: {stream1.conversation_id}\n---")
for chunk in stream1:
    print(chunk, end="", flush=True)
print("\n--- (assistant message persisted)")



Conversation ID: 62d8365610f742b9866db20f1c593487
---
Certainly! Step 2 refers to the concept of **Bandwidth and Quality** in streaming, which is crucial for delivering a seamless viewing or listening experience. Here’s a more detailed explanation:

### Bandwidth and Quality in Streaming

1. **Understanding Bandwidth**: Bandwidth is the maximum rate at which data can be transmitted over an internet connection. It is typically measured in megabits per second (Mbps). The higher the bandwidth, the more data can be sent and received simultaneously. This is particularly important for streaming, as video and audio files can be large and require significant data transfer.

2. **Impact on Streaming Quality**: The quality of the streamed content—such as resolution and clarity—depends heavily on the available bandwidth. For example:
   - **Standard Definition (SD)** video typically requires about 3 Mbps.
   - **High Definition (HD)** video (720p) requires around 5-8 Mbps.
   - **Full HD (1080p)